In [1]:
import os
import time
import math
# import pandas as pd
import numpy as np
from numbers import Number
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import visdom
from torch.autograd import Variable
from torch.utils.data import DataLoader
# import plotly.express as px
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import shutil

import lib.dist as dist
import lib.utils as utils
# import lib.datasets as dset
from lib.flows import FactorialNormalizingFlow

from elbo_decomposition import elbo_decomposition
from plot_latent_vs_true import plot_vs_gt_shapes, plot_vs_gt_faces


In [2]:
from vae_quant import VAE

In [3]:
# for filename in os.listdir('/home/desai.ven/TCVAE/img_align_celeba'):
#             # image = Image.open(os.path.join("images", filename))
#             image = Image.open(os.path.join('/home/desai.ven/TCVAE/img_align_celeba', filename))
#             image = transform(image)
#             self.dataset.append(image)

In [4]:
# root='/home/desai.ven/TCVAE/img_align_celeba'
# img_list=os.listdir(root)
# print(len(img_list))

In [5]:
# import pandas as pd

In [6]:
# att=pd.read_csv('/home/desai.ven/TCVAE/list_attr_celeba.csv')

In [7]:
# att.columns

In [8]:
# att=att[["image_id","Bald","Oval_Face","Male","Mustache"]]

In [9]:
# for i, (_,row) in enumerate(att.iterrows()):
#     if row["Bald"]==1:
#         shutil.copyfile("/home/desai.ven/TCVAE/img_align_celeba/"+row["image_id"], "/home/desai.ven/TCVAE/celeba_att/Bald/"+row["image_id"] )
#     if row["Oval_Face"]==1:
#         shutil.copyfile("/home/desai.ven/TCVAE/img_align_celeba/"+row["image_id"], "/home/desai.ven/TCVAE/celeba_att/Oval_Face/"+row["image_id"] )
#     if row["Male"]==1:
#         shutil.copyfile("/home/desai.ven/TCVAE/img_align_celeba/"+row["image_id"], "/home/desai.ven/TCVAE/celeba_att/Male/"+row["image_id"] )
#     if row["Mustache"]==1:
#         shutil.copyfile("/home/desai.ven/TCVAE/img_align_celeba/"+row["image_id"], "/home/desai.ven/TCVAE/celeba_att/Mustache/"+row["image_id"] )

In [10]:
# img_list=os.listdir("/home/desai.ven/TCVAE/celeba_att/Bald/")
# img_list.extend(os.listdir("/home/desai.ven/TCVAE/celeba_att/Oval_Face/"))
# img_list.extend(os.listdir("/home/desai.ven/TCVAE/celeba_att/Male/"))
# img_list.extend(os.listdir("/home/desai.ven/TCVAE/celeba_att/Mustache/"))

In [11]:
# print(len(img_list))

In [12]:
image_size = 64
pix_mean = [0.5, 0.5, 0.5]
pix_std = [0.5, 0.5, 0.5]

celeba_dataset = datasets.ImageFolder(
    root='/home/desai.ven/TCVAE/celeba_att/Bald',
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(pix_mean, pix_std),
    ]))

In [13]:
print(type(celeba_dataset))

<class 'torchvision.datasets.folder.ImageFolder'>


In [14]:
batch_size=32
num_worksers=4
# pin_memory= use_cuda
kwargs = {'num_workers': 4, 'pin_memory': True}

In [15]:
train_loader=torch.utils.data.DataLoader(
    celeba_dataset,
    batch_size=32,
    shuffle=True,
    **kwargs
)

In [16]:
type(celeba_dataset)

torchvision.datasets.folder.ImageFolder

In [17]:
prior_dist = dist.Normal()
q_dist = dist.Normal()

In [18]:
num_epochs=50
lr=1e-3 #4e-3
latent_dim=10
tcvae=True
conv= True
mss= False

log_freq=200
save="results_celeba_Bald"

In [19]:
vae = VAE(z_dim=latent_dim, use_cuda=True, prior_dist=prior_dist, q_dist=q_dist, include_mutinfo=True, tcvae=tcvae, conv=conv, mss=mss)

In [20]:
vae.beta=15
vae.lamb=0

In [21]:
optimizer = optim.Adam(vae.parameters(), lr=lr)
dataset_size = len(train_loader.dataset)
num_iterations = len(train_loader) * num_epochs
iteration = 0
train_elbo = []
    # initialize loss accumulator
elbo_running_mean = utils.RunningAverageMeter()
while iteration < num_iterations:
        for i, x in enumerate(train_loader):
            iteration += 1
            batch_time = time.time()
            vae.train()
            optimizer.zero_grad()
            # print(x[1])
            x=x[0]
            # print(x.shape)
            x = x.cuda()
            # wrap the mini-batch in a PyTorch Variable
            x = Variable(x)
            # do ELBO gradient and accumulate loss
            obj, elbo = vae.elbo(x, dataset_size)
            if utils.isnan(obj).any():
                raise ValueError('NaN spotted in objective.')
            obj.mean().mul(-1).backward()
            # elbo_running_mean.update(elbo.mean().data[0])
            elbo_running_mean.update(elbo.mean().data.item())
            optimizer.step()
            if iteration % log_freq == 0:
                train_elbo.append(elbo_running_mean.avg)
                print('[iteration %03d] time: %.2f \tbeta %.2f \tlambda %.2f training ELBO: %.4f (%.4f)' % (
                    iteration, time.time() - batch_time, vae.beta, vae.lamb,
                    elbo_running_mean.val, elbo_running_mean.avg))

                vae.eval()
                utils.save_checkpoint({'state_dict': vae.state_dict()}, save, 0)

vae.eval()
utils.save_checkpoint({'state_dict': vae.state_dict()}, save, 0)
            


[iteration 200] time: 0.02 	beta 15.00 	lambda 0.00 training ELBO: 55774.9375 (47466.2858)
[iteration 400] time: 0.02 	beta 15.00 	lambda 0.00 training ELBO: 52789.7344 (54236.0745)
[iteration 600] time: 0.02 	beta 15.00 	lambda 0.00 training ELBO: 51505.2656 (54071.0881)
[iteration 800] time: 0.02 	beta 15.00 	lambda 0.00 training ELBO: 57356.1133 (55615.7563)
[iteration 1000] time: 0.02 	beta 15.00 	lambda 0.00 training ELBO: 56316.9102 (55712.6610)
[iteration 1200] time: 0.02 	beta 15.00 	lambda 0.00 training ELBO: 53383.1172 (55404.3249)
[iteration 1400] time: 0.02 	beta 15.00 	lambda 0.00 training ELBO: 59231.2734 (56325.9183)
[iteration 1600] time: 0.02 	beta 15.00 	lambda 0.00 training ELBO: 57459.7578 (57686.4035)
[iteration 1800] time: 0.02 	beta 15.00 	lambda 0.00 training ELBO: 57177.4453 (56456.0760)
[iteration 2000] time: 0.02 	beta 15.00 	lambda 0.00 training ELBO: 63167.0664 (56166.9410)
[iteration 2200] time: 0.02 	beta 15.00 	lambda 0.00 training ELBO: 44831.8750 (5591

In [47]:
Test = datasets.ImageFolder(
    root='/home/desai.ven/TCVAE/Test/',
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(pix_mean, pix_std),
    ]))
    

In [33]:
# test_loader=torch.utils.data.DataLoader(
#     Test,
#     batch_size=1,
#     shuffle=True,
#     **kwargs
# )

In [69]:
# x=torch.tensor(Test[0])
x1=Test[0][0]
x2=Test[1][0]
x3=Test[2][0]
x4=Test[3][0]
x5=Test[4][0]

x_test=torch.stack([x1,x2,x3,x4],dim=0)
    

In [70]:
x_test=x_test.cuda()
# x_test = Variable(x_test)

In [73]:
_, reco_imgs, zs, _ = vae.reconstruct_img(x_test)
zs = zs[0:3]
batch_size, z_dim = zs.size()
xs = []
delta = torch.autograd.Variable(torch.linspace(-2, 2, 7), volatile=True).type_as(zs)
for i in range(z_dim):
    vec = Variable(torch.zeros(z_dim)).view(1, z_dim).expand(7, z_dim).contiguous().type_as(zs)
    vec[:, i] = 1
    vec = vec * delta[:, None]
    zs_delta = zs.clone().view(batch_size, 1, z_dim)
    zs_delta[:, :, i] = 0
    zs_walk = zs_delta + vec[None]
    xs_walk = vae.decoder.forward(zs_walk.view(-1, z_dim)).sigmoid()
    xs.append(xs_walk)
temp_3=torch.cat(xs, 0).data.cpu()

win_latent_walk =vutils.make_grid(temp_3,nrow=10, padding=2)
vutils.save_image(win_latent_walk, "latent_celeba/latent_2.png")

/tmp/ipykernel_2968/2832747731.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  delta = torch.autograd.Variable(torch.linspace(-2, 2, 7), volatile=True).type_as(zs)


In [46]:
x[0].shape

torch.Size([3, 64, 64])